In [ ]:
import numpy as np
import matplotlib.pylab as plt
import sys
sys.path.append('../zdrojaky')
from tsplot import tsplot
from statsmodels import tsa
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.api import qqplot

# Modelování nestacionární časové řady se sezónními vlivy

Nyní se podíváme na nestacionární časovou řadu se sezónností, konkrétně na již oblíbené letecké cestovatele (viz přednáška). Data jsou uložena v `international-airline-passengers.csv`.

**Úkol: natáhněte data (pozor na hlavičky) a analyzujte je pomocí funkce `tsplot()`. Jaké vlastnosti má časová řada?**

In [ ]:
fn = '../zdrojaky/international-airline-passengers.csv'
data = np.genfromtxt(fn, delimiter=',', usecols=1, skip_header=1)

In [ ]:
plt.figure(figsize=(15, 3))
tsplot(***)
plt.show()

**Úkol: diferencujte časovou řadu sezónně i nesezónně a analyzujte výsledky.**

In [ ]:
dt = data[***]-data[***] # D=1
***                      # d=1
tsplot()

**Úkol: Budeme fitovat pomocí třídy `SARIMAX` z `statsmodels.tsa.statespace.sarimax`. Ozkoušejte několik možných konfigurací parametrů - držte se pravidel z přednášky.**

In [ ]:
model = SARIMAX(data, order=(***,***,***), seasonal_order=(***,***,***,***))
result = model.fit()
print(result.summary())

- [Ljung-Boxův test testuje H0: no autocorrelation of residues](https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.acorr_ljungbox.html)
- [Heteroskedasticity test - H0: no heteroskedasticity](https://www.statsmodels.org/dev/generated/statsmodels.regression.recursive_ls.RecursiveLSResults.test_heteroskedasticity.html)
- [Jarque-Bera - H0: normální distribuce](https://www.statsmodels.org/dev/generated/statsmodels.stats.stattools.jarque_bera.html)

**Úkol: podívejme se nyní na ACF, PACF a řadu reziduí pomocí `tsplot()` a interpretujme výsledky.**

In [ ]:
plt.figure(figsize=(15,3))
tsplot(***)
plt.show()

**Úkol: vykreslíme histogram reziduí a qq-plot a interpretujeme výsledky.**

In [ ]:
plt.figure(figsize=(15,3))
ax = plt.subplot(1,2,1)
plt.hist(result.resid, bins=20)
plt.title('Histogram reziduí')
ax = plt.subplot(1,2,2)
qqplot(result.resid, line='q', ax=ax)
plt.show()

**Úkol: zkusme nyní predikce na `nsteps` dopředu. K tomu slouží `metoda result.get_forecast()`, která vrátí instanci obsahující mj. atribut `predicted_mean` (predikovaná hodnota) a `summary_frame()`, v němž najdeme kromě predikovaných hodnot i intervaly spolehlivosti. Vykresleme tedy predikce vč. intervalů.**

In [ ]:
nsteps = ***
forecast = result.get_forecast(steps=nsteps)
plt.figure(figsize=(15,3))
plt.plot(np.arange(data.size), data, label='observed')
plt.plot(np.arange(data.size, data.size+forecast.predicted_mean.size), 
         forecast.predicted_mean, 'r', label='predicted')
plt.fill_between(np.arange(data.size, data.size+forecast.predicted_mean.size), 
                 forecast.summary_frame()['mean_ci_lower'],
                 forecast.summary_frame()['mean_ci_upper'],
                 color='lightgray')
plt.legend()
plt.xlabel('month')
plt.ylabel('passengers')
plt.show()